In [11]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.autograd import Variable
from torch.nn import Parameter
from torch import Tensor
import torch.nn.functional as F
from util.loader import KddData
from origin.lstm import LSTMCell,LSTMModel

import math

cuda = True if torch.cuda.is_available() else False
input_dir = 'data/'
input_dim = 10
seq_dim = 10
num_class = 10

In [12]:
import torchmetrics

def runmodel(dataset, model):
    accuracy = torchmetrics.Accuracy()
    recall = torchmetrics.Recall(average='macro', num_classes=num_class)
    precision = torchmetrics.Precision(average='macro', num_classes=num_class)
    f1score = torchmetrics.F1(average='macro', num_classes=num_class)
    eval_loss = 0
    eval_acc = 0

    test_loader = dataset.test_dataloader
    for images, labels in test_loader:
        if torch.cuda.is_available():
            images = Variable(images.view(-1, seq_dim, input_dim).cuda())
        else:
            images = Variable(images.view(-1 , seq_dim, input_dim))
        # Forward pass only to get logits/output
        outputs = model(images)

        # Get predictions from the maximum value
        _, predicted = torch.max(outputs.data, 1)
        
        
        labels = labels.cpu()
        predicted = predicted.cpu()
        accuracy(predicted, labels)
        recall(predicted, labels)
        precision(predicted, labels)
        f1score(predicted, labels)

        acc = accuracy.compute().data.detach()
    print('Accuracy:',acc)

    rec = recall.compute().data.detach()
    prec = precision.compute().data.detach()
    f1sc = f1score.compute().data.detach()
    print(f" Recall: {rec}, precision: {prec}, f1score: {f1sc}")


testFile  = input_dir + 'test-wired-bin.csv'
testFile1 = input_dir + 'test-kr-bin.csv'
testFile2 = input_dir + 'test-us-bin.csv'
testFile3 = input_dir + 'test-wifi-bin.csv'
testFile4 = input_dir + 'test-4G-bin.csv'
testFile5 = input_dir + 'test-3G-bin.csv'

batch_size = 64

model = torch.load('origin/lstm.pth')
dataset = KddData(batch_size, testFile4)
runmodel(dataset, model)

/usr/local/lib/python3.8/dist-packages/torch/nn/functional.py:1805: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/usr/local/lib/python3.8/dist-packages/torch/nn/functional.py:1794: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Accuracy: tensor(0.6819)
 Recall: 0.5836828351020813, precision: 0.7927520275115967, f1score: 0.5453807711601257
